In [1]:
import sys
import os
import subprocess
from boto3.session import Session

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])
from utils import setup_cognito_user_pool, reauthenticate_user

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

sys.path[0]: D:\Code\github\Bedrock_agent
Region: ap-southeast-1


In [2]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool("Cognito_3LO_Google")
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: ap-southeast-1_a0Lucrb9n
Discovery URL: https://cognito-idp.ap-southeast-1.amazonaws.com/ap-southeast-1_a0Lucrb9n/.well-known/openid-configuration
Client ID: 129bkp4ighm57tmuipjnqbemv7
Bearer Token: eyJraWQiOiJzRGFBeDlPZWxXdnJCTDJhOWRMcHBNNnRvTGtndXFLcUQ1d3BERzFZdk1rPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI2OTVhYTVlYy0yMDgxLTcwZDItYWFlOC1hYTNjNWY5ZjljNjYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTEuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTFfYTBMdWNyYjluIiwiY2xpZW50X2lkIjoiMTI5YmtwNGlnaG01N3RtdWlwam5xYmVtdjciLCJvcmlnaW5fanRpIjoiNzI4ZTE2ZDEtOGZiOC00ZWM3LTllZWUtYTI5ZjE1ZWI4MTMxIiwiZXZlbnRfaWQiOiIzMzIwZmZmNy02MzAyLTQ5YzgtOTNjYS1kOGEwZDRmYzdhZGQiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIiwiYXV0aF90aW1lIjoxNzYyOTE4MzY4LCJleHAiOjE3NjI5MjE5NjgsImlhdCI6MTc2MjkxODM2OCwianRpIjoiNDY3NjM0YzgtYmE4ZC00OGYwLTk5YzMtN2RkNDY4OTdhMGQzIiwidXNlcm5hbWUiOiJ0ZXN0dXNlciJ9.Xz5zaEFsLfJGIrZMiiu1LBpMXS8TfLCGFzH7zblYFb0RnE50

In [4]:
provider_name = "google-cal-provider"

In [5]:
from bedrock_agentcore.services.identity import IdentityClient
import os
from dotenv import load_dotenv

load_dotenv()

google_client_id = os.environ["GOOGLE_CLIENT_ID"]
google_client_secret = os.environ["GOOGLE_CLIENT_SECRET"]

identity_client = IdentityClient(region)

# Configure Google OAuth2 provider - On-Behalf-Of User
google_provider = identity_client.create_oauth2_credential_provider({
    "name": provider_name,
    "credentialProviderVendor": "GoogleOauth2",
    "oauth2ProviderConfigInput": {
        "googleOauth2ProviderConfig": {
            "clientId": google_client_id,
            "clientSecret": google_client_secret
        }
    }
})
print(google_provider)
print("\n")
print(f"callbackUrl: {google_provider['callbackUrl']}")

{'ResponseMetadata': {'RequestId': 'a03b6e49-e1ea-4d2b-a6dc-888e729d2aa1', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 12 Nov 2025 09:21:12 GMT', 'content-type': 'application/json', 'content-length': '716', 'connection': 'keep-alive', 'x-amzn-requestid': 'a03b6e49-e1ea-4d2b-a6dc-888e729d2aa1', 'x-amzn-remapped-x-amzn-requestid': '4ffebb09-0bf0-4193-9bab-fed6088a71e1', 'x-amzn-remapped-content-length': '716', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'T7GtZHuySQ0EEIg=', 'x-amzn-trace-id': 'Root=1-69145188-50d3a8f87cb9ff5240246624', 'x-amzn-remapped-date': 'Wed, 12 Nov 2025 09:21:12 GMT'}, 'RetryAttempts': 3}, 'clientSecretArn': {'secretArn': 'arn:aws:secretsmanager:ap-southeast-1:566801649110:secret:bedrock-agentcore-identity!default/oauth2/google-cal-provider-Mz5h6N'}, 'name': 'google-cal-provider', 'credentialProviderArn': 'arn:aws:bedrock-agentcore:ap-southeast-1:566801649110:token-vault/default/oauth2credentialprovider/google-cal-provider', 'callbackUrl'

In [6]:
# Get the OAuth2 callback URL based on the current environment (notebook/SageMaker)
# This is evaluated HERE in the notebook, not in the agent container
from oauth2_callback_server import get_oauth2_callback_url
oauth2_callback_url_for_agent = get_oauth2_callback_url()

print(f"Callback URL for agent (determined from notebook environment): {oauth2_callback_url_for_agent}")

# Write strands_claude_google_3lo.py with the callback URL embedded as a string literal
google_agent_code = f'''import os
import datetime
import json
import asyncio
import traceback
from typing import Dict, Any, Optional, AsyncGenerator

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Required OAuth2 scope for Google Calendar API
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# Global variable to store the access token
google_access_token: Optional[str] = None


@tool(
    name="Get_calendar_events_today",
    description="Retrieves the calendar events for the day from your Google Calendar"
)
def get_calendar_events_today() -> str:
    """
    Retrieve calendar events for today from Google Calendar.

    Returns:
        str: JSON string containing events or error information
    """
    global google_access_token

    # Check if we already have a token
    if not google_access_token:
        return json.dumps({{
            "auth_required": True,
            "message": "Google Calendar authentication is required. Please wait while we set up the authorization.",
            "events": []
        }})

    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)

        # Calculate today's time range
        today_start = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)

        # Format with CDT timezone (-05:00)
        time_min = today_start.strftime('%Y-%m-%dT00:00:00-05:00')
        time_max = today_end.strftime('%Y-%m-%dT23:59:59-05:00')

        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=time_min,
                timeMax=time_max,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])

        if not events:
            return json.dumps({{"events": []}})

        return json.dumps({{"events": events}})

    except HttpError as error:
        error_message = str(error)
        return json.dumps({{"error": error_message, "events": []}})
    except Exception as e:
        error_message = str(e)
        return json.dumps({{"error": error_message, "events": []}})


# Initialize the agent with tools and your preferred model choice
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[get_calendar_events_today]
)

# Initialize app and streaming queue
app = BedrockAgentCoreApp()


class StreamingQueue:
    """A queue for streaming responses asynchronously."""

    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()

    async def put(self, item: str) -> None:
        """Add an item to the queue."""
        await self.queue.put(item)

    async def finish(self) -> None:
        """Mark the queue as finished."""
        self.finished = True
        await self.queue.put(None)

    async def stream(self) -> AsyncGenerator[str, None]:
        """Stream items from the queue."""
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item


queue = StreamingQueue()


async def on_auth_url(url: str) -> None:
    """Handle authorization URL callback."""
    print(f"Authorization url: {{url}}")
    await queue.put(f"Authorization url: {{url}}")


async def agent_task(user_message: str) -> None:
    """
    Execute the agent task with authentication handling.

    Args:
        user_message: The user's input message
    """
    try:
        await queue.put("Begin agent execution")

        # Call the agent first to see if it needs authentication
        response = agent(user_message)

        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)

        # Check if the response indicates authentication is required
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth",
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)

        if needs_auth:
            await queue.put("Authentication required for Google Calendar access. Starting authorization flow...")

            # Trigger the 3LO authentication flow
            try:
                global google_access_token
                google_access_token = await need_token_3lo_async(access_token='')
                await queue.put("Authentication successful! Retrying calendar request...")

                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"auth_error: {{auth_error}}")
                await queue.put(f"Authentication failed: {{str(auth_error)}}")

        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {{str(e)}}")
    finally:
        await queue.finish()


@requires_access_token(
    provider_name={provider_name},
    scopes=SCOPES,
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
    callback_url="{oauth2_callback_url_for_agent}"  # ← Callback URL determined from notebook environment
)
async def need_token_3lo_async(*, access_token: str) -> str:
    """
    Handle 3-legged OAuth token retrieval.

    Args:
        access_token: The OAuth access token

    Returns:
        str: The access token
    """
    global google_access_token
    google_access_token = access_token
    return access_token


@app.entrypoint
async def agent_invocation(payload: Dict[str, Any]) -> AsyncGenerator[str, None]:
    """
    Main entrypoint for agent invocations.

    Args:
        payload: The request payload containing the prompt

    Yields:
        str: Streaming response items
    """
    user_message = payload.get(
        "prompt",
        "No prompt found in input, please guide customer to create a json payload with prompt key"
    )

    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))

    # Return the stream, but ensure the task runs concurrently
    async def stream_with_task() -> AsyncGenerator[str, None]:
        # Stream results as they come
        async for item in queue.stream():
            yield item

        # Ensure the task completes
        await task

    return stream_with_task()


if __name__ == "__main__":
    app.run()
'''

# Write the file
with open("strands_claude_google_3lo.py", "w") as f:
    f.write(google_agent_code)

print("✅ strands_claude_google_3lo.py written successfully with embedded callback URL")

ImportError: cannot import name 'UserTokenIdentifier' from 'bedrock_agentcore.services.identity' (D:\Code\github\Bedrock_agent\venv\Lib\site-packages\bedrock_agentcore\services\identity.py)

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_google_3lo.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    memory_mode="NO_MEMORY",
    agent_name="strands_agent_google_3lo",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
print(response)

In [ ]:
!cat .bedrock_agentcore.yaml

In [ ]:
from oauth2_callback_server import get_oauth2_callback_url

# Deploy the agent to AgentCore Runtime and get deployment details
launch_result = agentcore_runtime.launch()
print(launch_result)

if launch_result.agent_id:
    # Extract the workload name from the deployed agent's ID for identity management
    workload_name = launch_result.agent_id
    # Retrieve the current workload identity configuration from AgentCore Identity
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    # Extract existing OAuth2 callback URLs that are already registered for this workload
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    # Get the local OAuth2 callback server URL for session binding (localhost:9090/oauth2/callback)
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    # Register the local callback URL with the workload identity to enable OAuth2 session binding
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
    print(updated_workload_identity)

In [ ]:
import json
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(f"Final status: {status}")

In [ ]:
from oauth2_callback_server import store_token_in_oauth2_callback_server, wait_for_oauth2_server_to_be_ready

bearer_token = reauthenticate_user(cognito_config.get("client_id"))

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # Start the OAuth2 callback server
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print("Failed to start OAuth2 callback server to handle session binding "
              "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)")
    else:
        store_token_in_oauth2_callback_server(bearer_token)
        invoke_response = agentcore_runtime.invoke(
            {"prompt": "What is in my agenda for today? Highlight the main events!"},
            bearer_token=bearer_token
        )
        print(invoke_response)
finally:
    oauth2_callback_server_process.terminate()

In [ ]:
from chatbot_app_cognito import get_streamlit_url

# Change to the current directory where the chatbot_app_cognito.py file is located
notebook_dir = os.getcwd()

# Start the Streamlit app
print("Starting Streamlit app...")

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    wait_for_oauth2_server_to_be_ready()

    # Run streamlit in the current directory
    process = subprocess.Popen([
        sys.executable, "-m", "streamlit", "run", "chatbot_app_cognito.py",
        "--server.port=8501", "--server.showEmailPrompt=false"
    ],
        cwd=notebook_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Print the output as it comes
    for line in iter(process.stdout.readline, ''):
        if line:
            if "8501" in line:
                print("\n🎉 Streamlit app is ready!")
                streamlit_url = get_streamlit_url()
                print(f'\n🚀 Streamlit Application URL:\n{streamlit_url}\n')
                print("⚠️ To stop the app, interrupt the kernel or press Ctrl+C in the terminal")
                break

except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    oauth2_callback_server_process.terminate()
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")